In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules  
from ast import literal_eval
from mlxtend.preprocessing import TransactionEncoder 

import warnings
warnings.filterwarnings("ignore")


In [24]:
df = pd.read_csv('customer_product_recommendation.csv')
df.head()

,Customer_ID,Age_Group,Gender,Income_Level,Marital_Status,Occupation,Shopping_Frequency,Purchase_History,Preferred_Brand,Recommended_Product
0,CUST_001,Senior,Male,Medium,Divorced,Professional,Moderate,"['Clothing', 'Beauty Products']",No Preference,Home Decor
1,CUST_002,Adult,Female,Low,Divorced,Student,Frequent,"['Electronics', 'Groceries']",No Preference,Home Decor
2,CUST_003,Adult,Female,High,Single,Retired,Moderate,"['Gaming Accessories', 'Fitness Equipment']",No Preference,Clothing
3,CUST_004,Adult,Female,High,Single,Retired,Moderate,"['Gaming Accessories', 'Groceries', 'Fitness E...",Brand_C,Books
4,CUST_005,Adult,Female,High,Married,Student,Frequent,"['Sports Gear', 'Gaming Accessories', 'Fitness...",No Preference,Books


In [25]:
df['Purchase_History'] = df['Purchase_History'].apply(literal_eval)

In [26]:

# Gabungkan 'Age_Group' dan 'Gender' ke dalam setiap transaksi
df['Transaction_items'] = df.apply(lambda data: [data['Age_Group'], data['Gender']] + data['Purchase_History'], axis=1)


In [27]:
# Konversi transaksi ke dalam format list
transaction = df['Transaction_items'].tolist()

# Gunakan TransactionEncoder
encoder = TransactionEncoder()
transData = encoder.fit_transform(transaction)

# Ubah hasil encoding menjadi DataFrame
transData = pd.DataFrame(transData, columns=encoder.columns_)

# Tampilkan DataFrame hasil encoding
print(transData.head())


   Adult  Beauty Products  Books  Clothing  Electronics  Female  \
0  False             True  False      True        False   False   
1   True            False  False     False         True    True   
2   True            False  False     False        False    True   
3   True            False  False     False        False    True   
4   True            False  False      True        False    True   

   Fitness Equipment  Gaming Accessories  Groceries  Home Decor   Male  \
0              False               False      False       False   True   
1              False               False       True       False  False   
2               True                True      False       False  False   
3               True                True       True        True  False   
4               True                True      False       False  False   

   Senior  Sports Gear   Toys  Young  
0    True        False  False  False  
1   False        False  False  False  
2   False        False  False  Fals

In [31]:
# find the association
frequentItem = apriori(transData, min_support=0.1, use_colnames=True)
rules = association_rules(frequentItem, metric='lift', min_threshold=0.5)
# Mencetak aturan asosiasi dengan kolom yang benar
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


            antecedents         consequents  support  confidence      lift
0               (Adult)   (Beauty Products)     0.13    0.371429  1.198157
1     (Beauty Products)             (Adult)     0.13    0.419355  1.198157
2               (Books)             (Adult)     0.11    0.323529  0.924370
3               (Adult)             (Books)     0.11    0.314286  0.924370
4               (Adult)          (Clothing)     0.13    0.371429  1.092437
..                  ...                 ...      ...         ...       ...
181   (Male, Groceries)             (Young)     0.10    0.454545  1.623377
182  (Young, Groceries)              (Male)     0.10    0.769231  1.569859
183              (Male)  (Young, Groceries)     0.10    0.204082  1.569859
184             (Young)   (Male, Groceries)     0.10    0.357143  1.623377
185         (Groceries)       (Male, Young)     0.10    0.222222  1.234568

[186 rows x 5 columns]
